In [1]:
# Import stuff
import pandas as pd
import numpy as np

In [2]:
# # Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# # ms-python.python added
# import os
# try:
# 	# os.chdir(os.path.join(os.getcwd(), '..'))
#     os.chdir(os.path.join(os.getcwd(), "./scripts-notebooks"))
#     print(os.getcwd())
# except Exception as e:
# 	print(e)

/Users/SamYang95/GRN_Inference_IY/scripts-notebooks


In [3]:
# Get expression data
ko_df = pd.read_csv('../dataset/yeast_networks/expression/KO.txt', sep='\t').set_index('Gene')
nv_df = pd.read_csv('../dataset/yeast_networks/expression/NatVar.txt', sep='\t').set_index('Gene')
stress_df = pd.read_csv('../dataset/yeast_networks/expression/Stress.txt', sep='\t').set_index('Name')

# Get Transcription Factors
ko_tf = pd.read_csv('../dataset/yeast_networks/expression/KO_TF_names.txt', sep='\t', header=None)
ko_tf.columns = ['TF']

nv_tf = pd.read_csv('../dataset/yeast_networks/expression/NatVar_TF_names.txt', sep='\t', header=None)
nv_tf.columns = ['TF']

stress_tf = pd.read_csv('../dataset/yeast_networks/expression/Stress_TF_names.txt', sep='\t', header=None)
stress_tf.columns = ['TF']

# Note that there are different gene list for each df, so we will use ko as test for now.

In [14]:
def merge_dataset(df_list, merge_method, index_name=None):
    '''
    Merge data frames and return a merged data frame
    Args:
        - df_list (required): list of data frames to merge
        - merge_method (required): a function that merges two data frames (see example below)
        - index_name (optional): index_name for the returned data frame
    Example Usage:
        merge_method_df = lambda x, y: x.merge(y, how='inner', left_index=True, right_index=True) # define merge_method
        df_all = merge_dataset([ko_df, nv_df, stress_df], merge_method=merge_method_df, index_name='Gene')
        
        merge_method_tf = lambda x, y: x.merge(y, how='inner', left_on='TF', right_on='TF')
        tf_all = merge_dataset([ko_tf, nv_tf, stress_tf], merge_method=merge_method_tf)
    '''
    if len(df_list) == 0: raise Exception("input list cannot be empty")

    df = df_list[0]
    for i in range(1, len(df_list)):
        df = merge_method(df, df_list[i])

    if index_name:
        df.index.name = index_name

    return df

In [5]:
merge_method_df = lambda x, y: x.merge(y, how='inner', left_index=True, right_index=True)
merge_method_tf = lambda x, y: x.merge(y, how='inner', left_on='TF', right_on='TF')

In [6]:
df_all = merge_dataset([ko_df, nv_df, stress_df], merge_method=merge_method_df, index_name='Gene')
tf_all = merge_dataset([ko_tf, nv_tf, stress_tf], merge_method=merge_method_tf)

In [8]:
df_all.to_csv('../dataset/yeast_networks/expression/All.txt', sep='\t')
tf_all.to_csv('../dataset/yeast_networks/expression/All_TF_names.txt', index=False, columns=['TF'], sep='\t', header=False)

## See if retrieved data is correct

In [10]:
df_retrieved = pd.read_csv('../dataset/yeast_networks/expression/All.txt', sep='\t').set_index('Gene')

tf_retrieved = pd.read_csv('../dataset/yeast_networks/expression/All_TF_names.txt', sep='\t', header=None)
tf_retrieved.columns = ['TF']

In [13]:
df_og = ko_df.merge(nv_df, how='inner', left_index=True, right_index=True)
df_og = df_og.merge(stress_df, how='inner', left_index=True, right_index=True)

tf_og = ko_tf.merge(nv_tf, how='inner', left_on='TF', right_on='TF')
tf_og = tf_og.merge(stressd_tf, how='inner', left_on='TF', right_on='TF')